In [39]:
import pandas as pd
import seaborn as sns
import plotly.express as px

In [40]:
df_train = pd.read_csv("/content/train_data.txt",sep=':::', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])
df_test=pd.read_csv("/content/test_data_solution.txt",sep=':::', names=['ID', 'TITLE','GENRE', 'DESCRIPTION'])

<ipython-input-40-9667ec753b43>:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.

<ipython-input-40-9667ec753b43>:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



In [41]:
df_train.head()

,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [42]:
df_train.drop('ID',axis=1,inplace=True)
df_test.drop('ID',axis=1,inplace=True)

In [43]:
px.bar(df_train['GENRE'].value_counts(),color=df_train['GENRE'].unique(), title='Number of movies in each genre')

In [44]:
#df_train=df_train.sample(frac=0.1)
#df_test= df_test.sample(frac=0.1)

In [45]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
# Download the tokenizer models if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
def text_clean(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'@\S+', '', text)  # Remove mentions
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'pic.\S+', '', text)  # Remove patterns starting with 'pic'
    text = re.sub(r"[^a-zA-Z+']", " ", text)  # Remove non-alphabetic characters (except + and ')
    text = re.sub(r'\b[a-zA-Z]\b', '', text)  # Remove single letters
    text = re.sub(r'[^\w\s]', '', text)  # Remove remaining punctuation
    text = ' '.join(text.split())  # Remove extra whitespace
    words = nltk.word_tokenize(text)
    stopwords_list = stopwords.words('english')
    words = [word for word in words if word not in stopwords_list and len(word) >=2]
    text = ' '.join(words)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [47]:
df_train['DESCRIPTION'] = df_train['DESCRIPTION'].apply(text_clean)
df_test['DESCRIPTION'] = df_test['DESCRIPTION'].apply(text_clean)

In [48]:
df_train.head()

,TITLE,GENRE,DESCRIPTION
0,Oscar et la dame rose (2009),drama,listening conversation doctor parents year old...
1,Cupid (1997),thriller,brother sister past incestuous relationship cu...
2,"Young, Wild and Wonderful (1980)",adult,bus empties students field trip museum natural...
3,The Secret Sin (1915),drama,help unemployed father make ends meet edith tw...
4,The Unrecovered (2007),drama,film title refers un recovered bodies ground z...


In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
x_train=tf.fit_transform(df_train['DESCRIPTION'])
x_test=tf.transform(df_test['DESCRIPTION'])

In [50]:
y_train=df_train['GENRE']
y_test=df_test['GENRE']

In [51]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()

In [52]:
nb.fit(x_train,y_train)

MultinomialNB()

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train,nb.predict(x_train))

0.46987862913638545

In [54]:
accuracy_score(y_test,nb.predict(x_test))

0.44446494464944647

In [55]:
def predict_genre(description):
  description=text_clean(description)
  description=tf.transform([description])
  return nb.predict(description)[0]

In [56]:
#predicting genre of Oppenheimer
description="the brilliant physicist who led the development of the atomic bomb during World War II as part of the top-secret Manhattan Project. The film dives deep into the personal and professional struggles of Oppenheimer, exploring his complex character as a scientist, his moral dilemmas, and the profound impact of his work on global history"
predict_genre(description)

' documentary '